# Necessary environment setting

In [ ]:
# Divine beast bless no bug here! 
#         ┌─┐    ┌─┐
#      ┌─┘ ┴───┘ ┴──┐
#      │                   │
#      │       ───       │
#      │  ─┬┘     └┬─  │
#      │                   │
#      │       ─┴─       │
#      │                   │
#      └─┐         ┌───┘
#          │         │
#          │         │
#          │         │
#          │         └──────────────┐
#          │                                  │
#          │                                  ├─┐
#          │                                  ┌─┘
#          │                                  │
#          └─┐  ┐  ┌──────┬──┐  ┌──┘
#            │  ─┤ ─┤         │  ─┤ ─┤
#            └──┴──┘         └──┴──┘

!pip install transformers
!pip install boto3
!git clone https://github.com/NVIDIA/apex
!pip install jiwer==2.2.0
!pip install -U nltk
%cd apex
!pip install -v --no-cache-dir ./
%cd .. 


     |████████████████████████████████| 2.3MB 2.9MB/s 
     |████████████████████████████████| 3.3MB 19.8MB/s 
     |████████████████████████████████| 901kB 33.8MB/s 
     |████████████████████████████████| 133kB 3.0MB/s 
     |████████████████████████████████| 81kB 6.9MB/s 
     |████████████████████████████████| 7.6MB 12.0MB/s 
     |████████████████████████████████| 143kB 50.3MB/s 
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.5 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Cloning into 'apex'...
remote: Enumerating objects: 8048, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 8048 (delta 65), reused 97 (delta 44), pack-reused 7913
Receiving obj

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
# mount the google drive to colab
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
# data preprocessing
from sklearn.model_selection import train_test_split
import pandas as pd
import re
import numpy as np
import json
import h5py

# control
import argparse
import logging
from tqdm import trange

# pytorch
import torch
print(torch.__version__)
import torch.nn.functional as F

# transformers
from transformers import GPT2Config
from transformers import AutoTokenizer, AutoModelWithLMHead
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import BartForCausalLM, BartTokenizer,BartConfig

1.8.1+cu101


/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


# data preprocessing

In [ ]:
# the original dataframe
df_ori = pd.read_csv("./gdrive/MyDrive/COMP0087/full_dataset.csv")

In [ ]:
np.random.seed(7)
train_len = 200000
test_len = 200
subset_idx = np.random.choice(range(len(df_ori)),train_len+test_len,replace=False)
train_idx = subset_idx[0:train_len]
test_idx = subset_idx[train_len:]
df = df_ori.loc[subset_idx ,:]
# df_test = df_ori.loc[subset_idx[-100:],:]

In [ ]:
# data preprocessing
remove1 = df.loc[df.title.map(lambda x: len(x)<4 )]
remove2 = df.loc[df.ingredients.map(lambda x: len(x)<2)]
remove3 = df.loc[df.directions.map(lambda x: len(x) < 2 or len(''.join(x)) < 30)]
remove4 = df.loc[df.directions.map(lambda x: re.search('(step|mix all)', ''.join(str(x)), re.IGNORECASE)!=None)]
len(remove3)+len(remove2)+len(remove1)+len(remove4)
df.drop(df[df.title.map(lambda x: len(x)<4)].index, inplace=True)
df.drop(df[df.ingredients.map(lambda x: len(x)<2)].index, inplace=True)
df.drop(df[df.directions.map(lambda x: len(x) < 2 or len(''.join(x)) < 30)].index, inplace=True)
df.drop(df[df.directions.map(lambda x: re.search('(step|mix all)', ''.join(str(x)), re.IGNORECASE)!=None)].index, inplace=True)

In [ ]:
df.drop(df[df.title.map(lambda x: len(x)<4)].index, inplace=True)
df.drop(df[df.ingredients.map(lambda x: len(x)<2)].index, inplace=True)
df.drop(df[df.directions.map(lambda x: len(x) < 2 or len(''.join(x)) < 30)].index, inplace=True)
df.drop(df[df.directions.map(lambda x: re.search('(step|mix all)', ''.join(str(x)), re.IGNORECASE)!=None)].index, inplace=True)

In [ ]:
df.reset_index(inplace=True)
train, test = train_test_split(df, test_size=0.05) #use 5% for test set
# we only want first 10000 and 100 for train/test
# this has an error: train[75400:75600]
# train = train[50000:400000]
test = test[0:100]

In [ ]:
def df_to_plaintext_file(input_df, output_file, train = True):
    print("Writing to", output_file)
    
    with open(output_file, 'w') as f:
        for index, row in input_df.iterrows():
            if index%100000==0:
                print(index)
                print(res)
            if type(row.NER)!=str:
                continue
            title = row.title
            directions = json.loads(row.directions)
            if len(directions) <= 1 and train:
              continue
            # print(len(directions))
            ingredients = json.loads(row.ingredients)
            ner = json.loads(row.NER)
            # print(ner)
            res = "<RECIPE_START> <INPUT_START> " + " <NEXT_INPUT> ".join(ner) + " <INPUT_END> <INGR_START> " + \
              " <NEXT_INGR> ".join(ingredients) + " <INGR_END> <INSTR_START> " + \
              " <NEXT_INSTR> ".join(directions) + " <INSTR_END> <TITLE_START> " + title + " <TITLE_END> <RECIPE_END>"
            # print(res)
            f.write("{}\n".format(res))

            

In [ ]:
df_to_plaintext_file(train, 'control_tokens_train.txt')
df_to_plaintext_file(test, 'control_tokens_test.txt',train = False)

Writing to control_tokens_train.txt
100000
<RECIPE_START> <INPUT_START> bread <NEXT_INPUT> Barrel cheese <NEXT_INPUT> sausage <NEXT_INPUT> eggs <NEXT_INPUT> milk <NEXT_INPUT> dry mustard <NEXT_INPUT> lemon pepper <NEXT_INPUT> cream of mushroom <NEXT_INPUT> milk <INPUT_END> <INGR_START> 8 slices bread, with crusts cut off <NEXT_INGR> 2 c. grated sharp Cracker Barrel cheese <NEXT_INGR> 2 lb. hot (bulk) sausage, well cooked and well drained <NEXT_INGR> 6 large eggs <NEXT_INGR> 2 1/2 c. milk <NEXT_INGR> 3/4 tsp. dry mustard <NEXT_INGR> 1 tsp. lemon pepper <NEXT_INGR> 1 can cream of mushroom, celery or chicken soup <NEXT_INGR> 1/2 c. milk <INGR_END> <INSTR_START> Place bread slices in bottom of baking dish. <NEXT_INSTR> Mix together eggs, milk, dry mustard and lemon pepper. <NEXT_INSTR> Place the sausage next (over bread), grated sharp cheese and then egg ingredients. Refrigerate overnight. <NEXT_INSTR> Next morning, turn oven to 300°. <NEXT_INSTR> Add cream of mushroom, celery or chicken s

In [ ]:
def init_tokenizer(): 
  tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
  # tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
  special_tokens = {
    "additional_special_tokens": [
        "<TITLE_START>",
        "<TITLE_END>",
        "<INSTR_START>",
        "<NEXT_INSTR>",
        "<INSTR_END>",
        "<INGR_START>",
        "<NEXT_INGR>",
        "<INGR_END>",
        "<RECIPE_START>",
        "<RECIPE_END>",
        "<INPUT_START>",
        "<INPUT_END>",
        "<NEXT_INPUT>"
      ]
  }
  tokenizer.add_special_tokens(special_tokens)
  return tokenizer

tokenizer = init_tokenizer()

In [ ]:
from collections import OrderedDict
train_size = 150000
test_size = 100


end_token_id = tokenizer.convert_tokens_to_ids(["<RECIPE_END>"])[0]
ing_token_id = tokenizer.convert_tokens_to_ids(["<INPUT_END>"])[0]

next_input_token_id = tokenizer.convert_tokens_to_ids(["<NEXT_INPUT>"])[0]

directions_size = 512
hf = h5py.File("unsupervised.h5", "w")
for filename in ["test", "train"]:
    out_np = []
    data = open("control_tokens_"+filename+".txt", "r")
    num = 0
    rows = 0
    last=[]
    for line in data:
        num+=1
        if num%10000 == 0:
            print("Process "+str(num)+"; Valid: "+str(rows))
        text_tokens = tokenizer(line)['input_ids']

        # generate the encoder input
        if ing_token_id in text_tokens:
          ing_idx = text_tokens.index(ing_token_id)
          temp_list = list(OrderedDict.fromkeys(text_tokens[3:ing_idx]))

          # if <NEXT_INPUT> not in list, do not need to process
          if next_input_token_id in temp_list:
            temp_list.remove(next_input_token_id)
          else:
            continue
          text_tokens = temp_list+text_tokens
        else:
          continue
        
        # error in one recipe
        if len(text_tokens) > directions_size or (50273 not in text_tokens): 
            continue

        # text_tokens_ids = tokenizer.convert_tokens_to_ids(text_tokens)
        text_tokens_ids = text_tokens

        # append <RECIPDE_END> token to the end
        while len(text_tokens_ids) < directions_size :
          text_tokens_ids.append(end_token_id)
        out_np.append(text_tokens_ids)
        rows+=1


        if rows == train_size and filename == 'train':
          print("training sample enough:",train_size)
          break
        if rows == test_size and filename == 'test':
          print("testing sample enough",test_size)
          break
    out_mat = np.matrix(out_np)
    print(out_mat.shape)
    hf.create_dataset(filename, data=out_mat)
hf.close()


(95, 512)


Token indices sequence length is longer than the specified maximum sequence length for this model (1092 > 1024). Running this sequence through the model will result in indexing errors


Process 10000; Valid: 9206
Process 20000; Valid: 18479
Process 30000; Valid: 27772
Process 40000; Valid: 36945
Process 50000; Valid: 46239
Process 60000; Valid: 55560
Process 70000; Valid: 64822
Process 80000; Valid: 74060
Process 90000; Valid: 83364
Process 100000; Valid: 92558
Process 110000; Valid: 101805
Process 120000; Valid: 111029
Process 130000; Valid: 120278
Process 140000; Valid: 129528
Process 150000; Valid: 138694
Process 160000; Valid: 147968
training sample enough: 150000
(150000, 512)


In [ ]:
tokenizer

PreTrainedTokenizer(name_or_path='facebook/bart-base', vocab_size=50265, model_max_len=1024, is_fast=False, padding_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True), 'additional_special_tokens': ['<TITLE_START>', '<TITLE_END>', '<INSTR_START>', '<NEXT_INSTR>', '<INSTR_END>', '<INGR_START>', '<NEXT_INGR>', '<INGR_END>', '<RECIPE_START>', 

In [ ]:
toke_result = tokenizer("<RECIPE_START> <INPUT_START> water <NEXT_INPUT> butter")
toke_result

{'input_ids': [0, 50273, 50275, 5412, 50277, 4297, 1334, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer.decode([50274])

'<RECIPE_END>'

# fine-tune with BART

In [ ]:
# fine-tuning model with bart
# logging info
import logging
logger = logging.getLogger(__name__)

import glob
import logging
import random
import gc
import boto3
import shutil

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset, SequentialSampler, RandomSampler
from torch.utils.data.distributed import DistributedSampler
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm, trange
import logging
import math
import os
from dataclasses import dataclass, field
from typing import Optional
from torch.utils.data import Dataset
import h5py
import torch

from transformers import WEIGHTS_NAME, AdamW, get_linear_schedule_with_warmup
from transformers import BartForCausalLM, BartTokenizer,BartConfig,BartForConditionalGeneration

In [ ]:
# data loading
class TextDataset(Dataset):
    def __init__(self, tokenizer, file_path='train', block_size=512):
        cached_features_file = "unsupervised.h5"

        logger.info("Loading features from cached file %s", cached_features_file)
        with h5py.File(cached_features_file, 'r') as f:
            if file_path=='test':
                self.examples = f[file_path][:] #this is a dev set, 10% of a test set
            else:
                self.examples = f[file_path][:]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item])

def load_and_cache_examples(args, tokenizer, evaluate=False):
    dataset = TextDataset(tokenizer, file_path="test" if evaluate else "train", block_size=args.block_size)
    print(dataset[0:5])
    return dataset

In [ ]:
parser = argparse.ArgumentParser()
## Required parameters
parser.add_argument("--train_data_file", default="unsupervised.h5", type=str, required=False,
                        help="The input training data file (a text file).")
parser.add_argument("--output_dir", default="./gdrive/MyDrive/COMP0087/Bart2_GRU_CP_0528_v1", type=str, required=False,
                        help="The output directory where the model predictions and checkpoints will be written.")

## Other parameters
parser.add_argument("--model_type", default="facebook/bart-base" ,type=str,   #"facebook/bart-base"
                        help="The model architecture to be fine-tuned.")
parser.add_argument("--model_name_or_path", default="facebook/bart-base", type=str,
                        help="The model checkpoint for weights initialization.")

parser.add_argument("--eval_data_file", default=None, type=str,
                        help="An optional input evaluation data file to evaluate the perplexity on (a text file).")

parser.add_argument("--config_name", default="", type=str,
                        help="Optional pretrained config name or path if not the same as model_name_or_path")
parser.add_argument("--tokenizer_name", default="", type=str,
                        help="Optional pretrained tokenizer name or path if not the same as model_name_or_path")
parser.add_argument("--cache_dir", default="", type=str,
                        help="Optional directory to store the pre-trained models downloaded from s3 (instread of the default one)")
parser.add_argument("--block_size", default=-1, type=int,
                        help="Optional input sequence length after tokenization."
                             "The training dataset will be truncated in block of this size for training."
                             "Default to the model max input length for single sentence inputs (take into account special tokens).")
parser.add_argument("--do_train", action='store_true',
                        help="Whether to run training.")
parser.add_argument("--do_eval", action='store_true',
                        help="Whether to run eval on the dev set.")
parser.add_argument("--evaluate_during_training", action='store_true',
                        help="Run evaluation during training at each logging step.")
parser.add_argument("--do_lower_case", action='store_true',
                        help="Set this flag if you are using an uncased model.")

parser.add_argument("--per_gpu_train_batch_size", default=8, type=int,
                        help="Batch size per GPU/CPU for training.")
parser.add_argument("--per_gpu_eval_batch_size", default=4, type=int,
                        help="Batch size per GPU/CPU for evaluation.")
parser.add_argument('--gradient_accumulation_steps', type=int, default=1,
                        help="Number of updates steps to accumulate before performing a backward/update pass.")
parser.add_argument("--learning_rate", default=5e-4, type=float,
                        help="The initial learning rate for Adam.")
parser.add_argument("--weight_decay", default=0.0, type=float,
                        help="Weight deay if we apply some.")
parser.add_argument("--adam_epsilon", default=1e-7, type=float,
                        help="Epsilon for Adam optimizer.")
parser.add_argument("--max_grad_norm", default=1.0, type=float,
                        help="Max gradient norm.")
parser.add_argument("--num_train_epochs", default=2, type=float,
                        help="Total number of training epochs to perform.")
parser.add_argument("--max_steps", default=-1, type=int,
                        help="If > 0: set total number of training steps to perform. Override num_train_epochs.")
parser.add_argument("--warmup_steps", default=0, type=int,
                        help="Linear warmup over warmup_steps.")

parser.add_argument('--logging_steps', type=int, default=50,
                        help="Log every X updates steps.")
parser.add_argument('--save_steps', type=int, default=50,
                        help="Save checkpoint every X updates steps.")
parser.add_argument("--eval_all_checkpoints", action='store_true',
                        help="Evaluate all checkpoints starting with the same prefix as model_name_or_path ending and ending with step number")
parser.add_argument("--no_cuda", action='store_true',
                        help="Avoid using CUDA when available")
parser.add_argument('--overwrite_output_dir', action='store_true',
                        help="Overwrite the content of the output directory")
parser.add_argument('--overwrite_cache', action='store_true',
                        help="Overwrite the cached training and evaluation sets")
parser.add_argument("--aws_bucket", default="", type=str,
                        help="Whether to upload to specified bucket.")
args, unknown = parser.parse_known_args()

def setup_args_for_model(args):
  args.model_type ="facebook/bart-base" # "facebook/bart-base"
  args.model_type = ""
  
  

In [ ]:
import random

def shuffle_instruction(this_batch,ins_element_id,recipe_end_id):
  end_index = (this_batch == recipe_end_id).nonzero()[0]
  # ins_element_id = tokenizer.convert_tokens_to_ids(["<NEXT_INSTR>"])[0]
  ing_index = (this_batch == ins_element_id).nonzero()
  ing_index = ing_index[ing_index<end_index]
  if len(ing_index) > 1:
    split_result = torch.tensor_split(this_batch, ing_index.squeeze())
    split_list = list(split_result[1:-1])
    random.shuffle(split_list)
    shuffle_batch = torch.cat((split_result[0],torch.cat(split_list),split_result[-1]))
    return shuffle_batch
  else: 
    return this_batch


def ins_token_idf(this_batch,ins_element_id,recipe_end_id):
  if (this_batch == recipe_end_id).nonzero().size()[0] == 0:
    return None
  end_index = (this_batch == recipe_end_id).nonzero()[0]
  ing_index = (this_batch == ins_element_id).nonzero()
  # print(torch.hstack((torch.full_like(ing_index,0),ing_index)))
  ing_index = ing_index[ing_index<end_index]
  if len(ing_index) > 0:
    return ing_index
  else: 
    return None


def split_ing_dirs(this_batch):
  token_list = tokenizer.convert_tokens_to_ids(["<INSTR_START>",
                                                "<INGR_START>",
                                                "<NEXT_INSTR>",
                                                "<RECIPE_END>",
                                                "<INPUT_END>",
                                                "<RECIPE_START>"])
  ins_start_id = token_list[0]
  ing_start_id = token_list[1]
  ins_element_id = token_list[2]
  recipe_end_id = token_list[3]
  input_end_id = token_list[4]
  recipe_start_id = token_list[5]

  sample = this_batch.clone()
  # ingredients size
  ing_size = 48
  ins_size = 512
  # the start index of ingredients
  # ing_index = (batch[i] == ing_start_id).nonzero()[0]
  # the start index of instruction
            
  # move the non-zero elements of ingredients vector to left
  ins_index = (sample == recipe_start_id).nonzero()[0]-1
  # print(ins_index)
  ingredients = torch.zeros(ing_size)
  # ingredients = torch.full_like(ingredients,input_end_id)
  # print(this_batch)
  if ins_index > ing_size:
    print("size overflow")
    ingredients[0:ing_size] = this_batch[0:ing_size]
  else:
    ingredients[0:ins_index] = this_batch[0:ins_index]
  sample[0:ins_index-1] = 0
  nz = sample.nonzero().squeeze()
  # move the non-zero elements of instructions vector to left
  directions = torch.zeros(sample.numel() - nz.numel())
  directions = torch.full_like(directions,recipe_end_id)
  # directions = this_batch[-1]
  directions = torch.cat((sample[nz], directions))
  directions[0] = 0
  # print(ins_subvector.shape)
  del sample
  return ingredients, directions


In [ ]:
import torch
import torch.nn as nn
# from transformer import AlbertTokenizer, AlbertForMultipleChoice
from transformers import BartForConditionalGeneration, BartTokenizer, BartConfig
from torch.nn import CrossEntropyLoss, MSELoss, GRU
from torch.nn.utils.rnn import pad_sequence


class bartWithGRU(nn.Module):
    def __init__(self, tokenizer, token_size=512):
        super(bartWithGRU, self).__init__()

        # device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
        self.bart = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
        self.bart.resize_token_embeddings(len(tokenizer))
        # self.lm_head = nn.Linear(self.bart.config.d_model, self.bart.config.vocab_size, bias=False)

        # add an module for consistency checking
        self.gru = torch.nn.GRU(self.bart.config.d_model, 32 , 1 ,batch_first = False)
        self.gru_head = nn.Linear(32, 1)
        self.linear_activation = nn.Sigmoid()
        self.cuda()

    def forward(self,encoder_batch, batch, inst_pos):
        # inst_pos: the position of token <NEXT_INSTR>, as a index list.
        # print(batch.shape)
        
        outputs = self.bart(input_ids = encoder_batch,
                            decoder_input_ids = batch,
                            output_hidden_states=True)
        last_hidden = outputs["decoder_hidden_states"][-1]
        # gru_input = torch.zeros(batch.size()[0],max_length,self.bart.config.d_model)
        tokens_list = []
        for i in range(batch.size()[0]):
          batch_hidden = last_hidden[inst_pos[i][:,0],inst_pos[i][:,1],:]
          tokens_list.append(batch_hidden)
        gru_input = pad_sequence(tokens_list)
        gru_output,_ = self.gru(gru_input)
        

        # print(ins_subvector.shape)
        # consist_output = self.consist_logit(ins_subvector.half().to(args.device))
        consist_output = self.gru_head(gru_output[-1])
        # print(consist_output.shape)
        consist_output = self.linear_activation(consist_output)

        logits = outputs["logits"]

        return logits, consist_output

In [ ]:
def save_model(args,model,tokenizer,model_class,tokenizer_class):
  # Create output directory if neede
  if not os.path.exists(args.output_dir):
    os.makedirs(args.output_dir)

  logger.info("Saving model checkpoint to %s", args.output_dir)
  # Save a trained model, configuration and tokenizer using `save_pretrained()`.
  # They can then be reloaded using `from_pretrained()`
  model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
  model_to_save.save_pretrained(args.output_dir)
  tokenizer.save_pretrained(args.output_dir)

  # Good practice: save your training arguments together with the trained model
  torch.save(args, os.path.join(args.output_dir, 'training_args.bin'))

  # Load a trained model and vocabulary that you have fine-tuned
  model = model_class.from_pretrained(args.output_dir)
  tokenizer = tokenizer_class.from_pretrained(args.output_dir, do_lower_case=args.do_lower_case)
  # model.to(args.device)


In [ ]:
from torch.nn import CrossEntropyLoss, BCELoss

def train_seq(args, train_dataset, model, tokenizer):
    """ Train the model """

    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)
    train_sampler = RandomSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size)

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
         'weight_decay': args.weight_decay},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args.warmup_steps,
                                                num_training_steps=t_total)

    try:
        from apex import amp
    except ImportError:
        raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")

    model, optimizer = amp.initialize(model, optimizer, opt_level="O2")

    # Train!
    global_step = 0
    epoch = 0
    tr_loss, logging_loss = 0.0, 0.0
    model.zero_grad()

    train_iterator = trange(int(args.num_train_epochs), desc="Epoch", disable=True)
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=True)
        
        alpha = 1.3
        lmbda = 0.5
        if epoch == 1:
          lmbda = 1.5
          # print("start discriminative part")
          # second step: fix the discriminative part 
          fix_layers = [model.gru, model.gru_head, model.linear_activation]
                # mybart.units_logit, mybart.units_activation]
          for layer in fix_layers:
            for parameter in layer.parameters():
              parameter.requires_grad = False


        epoch +=1
        for step, batch in enumerate(epoch_iterator):

            batch_drop = 0
            token_list = tokenizer.convert_tokens_to_ids(["<INSTR_START>",
                                                          "<INGR_START>",
                                                          "<NEXT_INSTR>",
                                                          "<RECIPE_END>"])
            ins_start_id = token_list[0]
            ing_start_id = token_list[1]
            ins_element_id = token_list[2]
            recipe_end_id = token_list[3]

            random_shuffle = torch.randint(2, (batch.size()[0], 1),device = args.device)
            # print(random_shuffle)
            
            # shuffle_instruction(batch[1],ins_element_id,recipe_end_id).shape

            # ins_tokens_list = []
            tokens_list = []
            max_length = 0

            encoder_batch = torch.empty(0,48)
            decoder_batch = torch.empty(0,512)
            for batch_no in range(len(batch)):
              
              if (batch[batch_no] == ins_start_id).nonzero().size()[0] == 0:
                print("An error happens, break")
                batch_drop = 1
                break

              if random_shuffle[batch_no] == 1:
                shuffle_batch = shuffle_instruction(batch[batch_no],
                                                      ins_start_id,ing_start_id)
              else:
                shuffle_batch = batch[batch_no]
              
              # split the encoder & decoder input
              encoder_input, decoder_input = split_ing_dirs(shuffle_batch)
              
              # encoder & decoder input list
              encoder_batch = torch.cat((encoder_batch, encoder_input.unsqueeze(0)))
              decoder_batch = torch.cat((decoder_batch, decoder_input.unsqueeze(0)))
                
              ins_pos = ins_token_idf(batch[batch_no],
                                      ins_element_id, recipe_end_id)
              if ins_pos == None:
                batch_drop = 1
                break

              # reshape for stacking
              ins_pos = ins_pos.reshape(-1,1)

              if ins_pos.size()[0] > max_length:
                max_length = ins_pos.size()[0]
              ins_idx = torch.hstack((torch.full_like(ins_pos,batch_no),ins_pos))
              tokens_list.append(ins_idx)
              
            if batch_drop == 1:
              print("An error in this batch, break")
              torch.cuda.empty_cache()
              continue

            # prepare the batch for GPU training
            encoder_batch = encoder_batch.long().to(args.device)
            decoder_batch = decoder_batch.long().to(args.device)
            inputs, labels = (decoder_batch[:, 0:-1], decoder_batch[:, 1:])
            inputs = inputs.to(args.device)
            labels = labels.to(args.device)

            # print(encoder_batch[0,0:15])
            # print(decoder_batch[0,0:15])
            # print(inputs[0,0:15])
            # print(labels[0,0:15])

            model.train()

            loss_CE = CrossEntropyLoss(reduction = "sum")
            loss_BCE = BCELoss()

            # model feed-forward
            logits, consist_output = model(encoder_batch, inputs, tokens_list)



            # loss adjustment
            bart_vocab_size = 50278
            loss1 = 0
            loss2 = 0

            # hyperparameter in multi-task training
            
            # print(logits.shape, labels.shape)

            for i in range(len(batch)):
              ing_end_index = (inputs[i] == ins_start_id).nonzero()[0]
              # print(logits[i,:ing_end_index].reshape(-1, bart_vocab_size).shape)
              # print(labels[i,:ing_end_index].reshape(-1).shape)
              loss1 += alpha * loss_CE(logits[i,:ing_end_index].reshape(-1, bart_vocab_size), 
                               labels[i,:ing_end_index].reshape(-1))
              loss1 += loss_CE(logits[i,ing_end_index:].reshape(-1, bart_vocab_size), 
                               labels[i,ing_end_index:].reshape(-1))

            loss1 = loss1/(len(batch)*511)

            loss2 = loss_BCE(consist_output, random_shuffle.float())
            loss = loss1 + lmbda * loss2

            if global_step % 100 == 0:
              logger.info(step)
              print("step", global_step)
              print("loss:",loss.item())

            if global_step % 10000 == 0 and global_step !=0:
              model_checkpoint = model.bart
              model_class = BartForConditionalGeneration
              tokenizer_class = BartTokenizer
              save_model(args,model_checkpoint,tokenizer,model_class,tokenizer_class)

            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            with amp.scale_loss(loss, optimizer) as scaled_loss:
                # print(scaled_loss)
                scaled_loss.backward()

            tr_loss += loss.item()
            if (step) % args.gradient_accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                # if args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    # Log metrics
                    # tb_writer.add_scalar('lr', scheduler.get_lr()[0], global_step)
                    # tb_writer.add_scalar('loss', (tr_loss - logging_loss) / args.logging_steps, global_step)
                    # logging_loss = tr_loss

            if args.max_steps > 0 and global_step > 2:
                epoch_iterator.close()
                break
            del inputs, labels,logits,consist_output , loss1, loss2, loss, encoder_batch, decoder_batch
            torch.cuda.empty_cache()
            # break

    return global_step, tr_loss / global_step, batch

In [ ]:
def model_init(args,logger, model_class, tokenizer):
  if args.eval_data_file is None and args.do_eval:
    raise ValueError("Cannot do evaluation without an evaluation data file. Either supply a file to --eval_data_file "
                         "or remove the --do_eval argument.")
  if os.path.exists(args.output_dir) and os.listdir(args.output_dir) and args.do_train and not args.overwrite_output_dir:
    raise ValueError("Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(args.output_dir))
  device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
  args.n_gpu = torch.cuda.device_count()
  args.device = device

  # Setup logging
  logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                        datefmt = '%m/%d/%Y %H:%M:%S',
                        level = logging.INFO)

  model = bartWithGRU(tokenizer)

  if args.block_size <= 0:
    args.block_size = tokenizer.max_len_single_sentence  # Our input block size will be the max possible for the model
  args.block_size = min(args.block_size, tokenizer.max_len_single_sentence)
  model.to(args.device)

  logger.info("Training/evaluation parameters %s", args)
  return model, logger

In [ ]:
torch.manual_seed(7)
train_dataset = load_and_cache_examples(args, tokenizer, evaluate=False)
mybart, logger = model_init(args,logger,BartForConditionalGeneration,tokenizer)
global_step, tr_loss, batch = train_seq(args, train_dataset, mybart, tokenizer)

tensor([[38299,    29, 20115,  ..., 50274, 50274, 50274],
        [11613,  4104, 10332,  ..., 50274, 50274, 50274],
        [14447,  4104, 22781,  ..., 50274, 50274, 50274],
        [ 1610,  4550, 24571,  ..., 50274, 50274, 50274],
        [  261,  1499, 14189,  ..., 50274, 50274, 50274]])


05/31/2021 10:40:21 - INFO - filelock -   Lock 139914556946832 acquired on /root/.cache/huggingface/transformers/f5310d276a6d1648d00c32fadc8bf7b4607e0fbd5b404fc4a0045960aa2bdfdb.3c4e938c08a46506ff9a4be12e0f858cd714b9a5c76090d571e7a4aa95cae853.lock


05/31/2021 10:40:22 - INFO - filelock -   Lock 139914556946832 released on /root/.cache/huggingface/transformers/f5310d276a6d1648d00c32fadc8bf7b4607e0fbd5b404fc4a0045960aa2bdfdb.3c4e938c08a46506ff9a4be12e0f858cd714b9a5c76090d571e7a4aa95cae853.lock


05/31/2021 10:40:23 - INFO - filelock -   Lock 139914509132496 acquired on /root/.cache/huggingface/transformers/486355ec722ef05fd480e999d4c763be56549ae930f6a3742ee721a5d2a05647.9faea28a6782a9589c09b1942c039943df02232d83d2ac288a69ddfa928eae22.lock


05/31/2021 10:40:33 - INFO - filelock -   Lock 139914509132496 released on /root/.cache/huggingface/transformers/486355ec722ef05fd480e999d4c763be56549ae930f6a3742ee721a5d2a05647.9faea28a6782a9589c09b1942c039943df02232d83d2ac288a69ddfa928eae22.lock


...

step 36800
loss: 1.6204910278320312
size overflow


05/31/2021 17:40:38 - INFO - __main__ -   18620


step 36900
loss: 1.6905627250671387
size overflow


05/31/2021 17:41:46 - INFO - __main__ -   18720


step 37000
loss: 2.071495532989502


## evaluation

In [ ]:
args.output_dir = "./gdrive/MyDrive/COMP0087/Bart2_GRU_CP_0528v2"
model = mybart.bart
model_class = BartForConditionalGeneration
tokenizer_class = BartTokenizer
save_model(args,model,tokenizer,model_class,tokenizer_class)

05/31/2021 18:15:37 - INFO - __main__ -   Saving model checkpoint to ./gdrive/MyDrive/COMP0087/Bart2_GRU_CP_0528v2


# generation

In [ ]:
raw_text = 'tomato,egg'
prepared_input = ' <RECIPE_START> <INPUT_START> ' + raw_text.replace(',', ' <NEXT_INPUT> ').replace(';', ' <INPUT_END>')
tokenizer.encode(prepared_input)[0:-1]

[0, 50273, 50275, 33063, 3938, 50277, 38299]

In [ ]:
def set_seed(args):
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)


def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float('Inf')):
    """ Filter a distribution of logits using top-k and/or nucleus (top-p) filtering
        Args:
            logits: logits distribution shape (vocabulary size)
            top_k > 0: keep only top k tokens with highest probability (top-k filtering).
            top_p > 0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
                Nucleus filtering is described in Holtzman et al. (http://arxiv.org/abs/1904.09751)
        From: https://gist.github.com/thomwolf/1a5a29f6962089e871b94cbd09daf317
    """
    assert logits.dim() == 1  # batch size 1 for now - could be updated for more but the code would be less clear
    top_k = min(top_k, logits.size(-1))  # Safety check
    if top_k > 0:
        # Remove all tokens with a probability less than the last token of the top-k
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probs > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        indices_to_remove = sorted_indices[sorted_indices_to_remove]
        logits[indices_to_remove] = filter_value
    return logits


def sample_sequence(model, length, raw_text, tokenizer, num_samples=1, temperature=1, top_k=0, top_p=0.0, device='cpu'):
    end_token = tokenizer.convert_tokens_to_ids(["<RECIPE_END>"])[0]
    ing_token_id = tokenizer.convert_tokens_to_ids(["<INPUT_END>"])[0]
    input_token = tokenizer.convert_tokens_to_ids(["<NEXT_INPUT>"])[0]

    prepared_input = ' <RECIPE_START> <INPUT_START> ' + raw_text.replace(',', ' <NEXT_INPUT> ').replace(';', ' <INPUT_END>')
    # print(prepared_input)

    context_tokens = tokenizer.encode(prepared_input)[:-1]
    context_len = len(context_tokens)
    
    encoder_tokens = torch.tensor(context_tokens[3:], dtype=torch.long, device=device)
    encoder_tokens = encoder_tokens[encoder_tokens != input_token]

    context = torch.tensor(context_tokens, dtype=torch.long, device=device)
    # the encoder condition
    raw_text_tokens = encoder_tokens
    context = context.unsqueeze(0).repeat(num_samples, 1)
    raw_text_tokens = raw_text_tokens.unsqueeze(0).repeat(num_samples, 1)
    
    # genereated token
    start_token =  tokenizer.convert_tokens_to_ids(["<RECIPE_START>"])[0]
    # generated = torch.tensor(start_token, dtype=torch.long, device=device).reshape(1).unsqueeze(0)
    generated = context
    # print(generated.shape)
    
    with torch.no_grad():
        for _ in range(length):
            # inputs = {'input_ids': generated}
            outputs = model(input_ids = raw_text_tokens, decoder_input_ids = generated)
            # outputs = model(**inputs)  # Note: we could also use 'past' with GPT-2/Transfo-XL/XLNet (cached hidden-states)
            # print(outputs)
            next_token_logits = outputs["logits"][0, -1, :] / temperature
            
            filtered_logits = top_k_top_p_filtering(next_token_logits, top_k=top_k, top_p=top_p)
            next_token = torch.multinomial(F.softmax(filtered_logits, dim=-1), num_samples=1)
            generated = torch.cat((generated, next_token.unsqueeze(0)), dim=1)
            #print(next_token)
            if next_token.item() == end_token:
                break
    return generated, context_len

In [ ]:
# raw_text = args.prompt if args.prompt else input("Comma-separated ingredients, semicolon to close the list >>> ")
model.to("cuda")
raw_text = 'potato,beef;'

out,context_len = sample_sequence(
            model=model,
            raw_text=raw_text,
            tokenizer=tokenizer,
            length=512,
            device = "cuda"
        )
out = out[0, context_len:].tolist()
text = tokenizer.decode(out, clean_up_tokenization_spaces=True)
if "<RECIPE_END>" not in text:
  print(text)
  print("Failed to generate, recipe's too long")
  full_text = prepared_input + text
  print(full_text)
else:
  full_text = prepared_input + text
  print(full_text)
  markdown = re.sub("<RECIPE_(START|END)>", "", full_text)
  recipe_n_title = markdown.split("<TITLE_START>")
  # title = "# " + recipe_n_title[1].replace("<TITLE_END>", "") + " #\n"
  markdown = recipe_n_title[0].replace("<INPUT_START>", "## Input ingredients ##\n`").replace("<INPUT_END>", "`\n")
  markdown = markdown.replace("<NEXT_INPUT>", "`\n`").replace("<INGR_START>", "## Ingredients ##\n* ").replace("<NEXT_INGR>", "\n* ").replace("<INGR_END>", "\n")
  markdown = markdown.replace("<INSTR_START>", "## Instructions ##\n1) ").replace("<NEXT_INSTR>", "\n1) ").replace("<INSTR_END>", "\n")
  markdown = re.sub("$ +#", "#", markdown)
  markdown = re.sub("( +`|` +)", "`", markdown)
  print(markdown)#


 <RECIPE_START> <INPUT_START> tomato <NEXT_INPUT> egg<INGR_START> 1 potato <NEXT_INGR> 1 bun beef <INGR_END> <INSTR_START> Bake beef for 2 hours at 375. <NEXT_INSTR> Chooke up.ALT. <NEXT_INSTR> Stuff up moon puffed bag onto cooked olives. <INSTR_END> <TITLE_START> Steak Bake Stuffed Walnut Caps <TITLE_END> <RECIPE_END>
  ## Input ingredients ##
`tomato`
`egg## Ingredients ##
*  1 potato 
*  1 bun beef 
 ## Instructions ##
1)  Bake beef for 2 hours at 375. 
1)  Chooke up.ALT. 
1)  Stuff up moon puffed bag onto cooked olives. 
 


In [ ]:
# gold standard
#df_eval = gold_st
model.to(args.device)
test_input = test.reset_index()
df_eval_ner = test_input["NER"]  #NER
df_eval_dir = test_input["directions"]

In [ ]:
def get_raw_input(NER):
    test_str = NER
    test_str = test_str.replace("[","")
    test_str = test_str.replace("]","")
    test_str = test_str.replace("\"","")

    return test_str

def get_instr(markdown):
  markdown = markdown.split("\n")
  if ' ## Instructions ##' in markdown:
    instr_index = markdown.index(' ## Instructions ##')
    
    output = [i for i in markdown[instr_index+1:]]

    if " " in output:
        output.remove(" ")

    if "" in output:
        output.remove("")    
    
    return output

  elif '## Instructions ##' in markdown:
    instr_index = markdown.index('## Instructions ##')

    output = [i for i in markdown[instr_index+1:]]

    if " " in output:
        output.remove(" ")
        
    if "" in output:
        output.remove("")    

    return output
    
  else:
    return ["failed to generate"]


def generate_recipe(raw_text):
    prepared_input = ' <RECIPE_START> <INPUT_START> ' + raw_text.replace(',', ' <NEXT_INPUT> ').replace(';', ' <INPUT_END>')
    # print(prepared_input)

    out, context_len = sample_sequence(
            model=model,
            raw_text=raw_text,
            tokenizer=tokenizer,
            length=512,
            device = "cuda"
        )
    out = out[0, context_len:].tolist()
    text = tokenizer.decode(out, clean_up_tokenization_spaces=True)
    if "<RECIPE_END>" not in text:
      print("Failed to generate, recipe's too long")
      full_text = prepared_input + text
      # print(full_text)
      return generate_recipe(raw_text)
    else:
      full_text = prepared_input + text
      # print(full_text)
      markdown = re.sub("<RECIPE_(START|END)>", "", full_text)
      recipe_n_title = markdown.split("<TITLE_START>")
      if len(recipe_n_title)<=1:
        return generate_recipe(raw_text)
      title = "# " + recipe_n_title[1].replace("<TITLE_END>", "") + " #\n"
      markdown = recipe_n_title[0].replace("<INPUT_START>", "## Input ingredients ##\n`").replace("<INPUT_END>", "`\n")
      markdown = markdown.replace("<NEXT_INPUT>", "`\n`").replace("<INGR_START>", "## Ingredients ##\n* ").replace("<NEXT_INGR>", "\n* ").replace("<INGR_END>", "\n")
      #markdown = markdown.replace("<INSTR_START>", "## Instructions ##\n\"").replace("<NEXT_INSTR>", "\n\"").replace("<INSTR_END>", "\n")
      markdown = markdown.replace("<INSTR_START>", "## Instructions ##\n").replace("<NEXT_INSTR>", "\n").replace("<INSTR_END>", "\n")
      markdown = re.sub("$ +#", "#", markdown)
      markdown = re.sub("( +`|` +)", "`", markdown)
  
      return markdown

In [ ]:
df_eval_dir

0     ["Place raspberries in a small bowl; mash with...
1     ["Whisk sour cream together with mustard in a ...
2     ["Mix together and pour over meat.", "Roast fo...
3     ["Saute onion, olives, capers and bacon or ham...
4     ["Mix brownies according to the directions on ...
                            ...                        
95    ["Blend together both of the flours and set as...
96    ["Shake one or two drops of bitters into a coc...
97    ["Wash each of the five varieties of lettuce i...
98    ["In a large bowl, toss together the apples an...
99    ["Dissolve yeast in warm water in warmed bowl....
Name: directions, Length: 100, dtype: object

In [ ]:
torch.manual_seed(1)
# generate recipes derived from gold standard ones
test_size = 100
replicated_size = 10
directions =  [[] for i in range(test_size)]

for i in range(test_size):
  raw_text = get_raw_input(df_eval_ner[i])
  print("\n Generating the recipe: ",i)

  for j in range(replicated_size):
    print("recipe: ",j)
    md = generate_recipe(raw_text)
    directions[i].append(get_instr(md))


 Generating the recipe:  0
recipe:  0
recipe:  1
recipe:  2
recipe:  3
recipe:  4
recipe:  5
recipe:  6
recipe:  7
recipe:  8
recipe:  9

 Generating the recipe:  1
recipe:  0
recipe:  1
recipe:  2
recipe:  3
recipe:  4
recipe:  5
recipe:  6
recipe:  7
recipe:  8
recipe:  9

 Generating the recipe:  2
recipe:  0
recipe:  1
recipe:  2
recipe:  3
recipe:  4
recipe:  5
recipe:  6
recipe:  7
recipe:  8
recipe:  9

 Generating the recipe:  3
recipe:  0
recipe:  1
recipe:  2
recipe:  3
recipe:  4
recipe:  5
recipe:  6
recipe:  7
recipe:  8
recipe:  9

 Generating the recipe:  4
recipe:  0
recipe:  1
recipe:  2
recipe:  3
recipe:  4
recipe:  5
recipe:  6
recipe:  7
recipe:  8
recipe:  9

 Generating the recipe:  5
recipe:  0
recipe:  1
recipe:  2
recipe:  3
recipe:  4
recipe:  5
recipe:  6
recipe:  7
recipe:  8
recipe:  9

 Generating the recipe:  6
recipe:  0
recipe:  1
recipe:  2
recipe:  3
recipe:  4
recipe:  5
recipe:  6
recipe:  7
recipe:  8
recipe:  9

 Generating the recipe:  7
recipe

In [ ]:
#func of cos_sim
import re
import math
from collections import Counter


def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    word = re.compile(r'\w+')
    words = word.findall(text)
    return Counter(words)


def get_result(content_a, content_b):
    text1 = content_a
    text2 = content_b

    vector1 = text_to_vector(text1)
    vector2 = text_to_vector(text2)

    cosine_result = get_cosine(vector1, vector2)
    return cosine_result

In [ ]:
#cosine_similarity

def COS_SIM(df_eval_dir,directions):
  """
  df_eval_dir := gold_standard recipes
  directions  := corresponding recipes """

  avg = 0

  for i in range(len(directions)):
    best = 0
    for j in range(len(directions[i])):
      cos = get_result(" ".join(eval(df_eval_dir[i])),
                      " ".join(directions[i][j]))
      best = max(best, cos)

    avg += best

  avg = avg/len(directions)

  print("avg:", avg)

In [ ]:
COS_SIM(df_eval_dir,directions)

avg: 0.5311078449518194


In [ ]:
import nltk
import nltk.translate.bleu_score as bleu
from nltk.translate.bleu_score import SmoothingFunction

import nltk.translate.gleu_score as gleu
import nltk.translate.meteor_score as meteor

In [ ]:
# Helper Func.
def list_to_words(recipe):
  words = []
  for i in recipe:
    words += i.split()

  return words

# New BLEU/GLEU
def bleu_score(recipe, refer):
    hyp = list_to_words(eval(recipe))
    refs = []
    for i in refer:
        # print(list_to_words(i))
        refs.append(list_to_words(i))

    smoothie = SmoothingFunction().method5
    score_ref_a = bleu.sentence_bleu(refs, hyp, smoothing_function=smoothie, weights=(1,0,0,0))
    return score_ref_a

def gleu_score(recipe, refer):
    hyp = list_to_words(eval(recipe))
    refs = []
    for i in refer:
        refs.append(list_to_words(i))

    score_ref_a = gleu.sentence_gleu(refs, hyp)
    return score_ref_a

In [ ]:
bleu_avg = []
gleu_avg = []

for i in range(len(directions)):
  # print(bleu_score(df_eval_dir[i], directions[i]))
  bleu_avg.append(bleu_score(df_eval_dir[i], directions[i]))

for i in range(len(directions)):
  gleu_avg.append(gleu_score(df_eval_dir[i], directions[i]))
print(np.mean(bleu_avg))
print(np.mean(gleu_avg))

0.7885782453010589
0.08522751470462167


In [ ]:
!sudo apt-get install git-lfs
!pip install huggingface_hub
!transformers-cli login
!git config --global user.email "uclqjia@ucl.ac.uk"
!git config --global user.name "jky594176"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 2s (880 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-pre

2021-05-31 19:39:14.618209: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0

        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        
Username: jky594176
Password: 
Login successful
Your token: wFlaWwkzYsLYQJZVfUieRigrdDLeDnTTZyiVhPAaIgeOELnnojbdQUAzHcSqpkgfkfKmxjkzOAmXBRTuYACjeYBoMzLFYVQgxRPnCXlpkACwUKifzsfRoihlcNWIGVgn 

Your token has been saved to /root/.huggingface/token


In [ ]:
model_type = ["recipe_GPT2","recipe_BART1","recipe_BART2","recipe_BART1_NN","recipe_BART1_GRU","BART2_GRU"]
# output_dir = "./gdrive/MyDrive/COMP0087/Bart1Gru_CP_350k"
# model = BartForConditionalGeneration.from_pretrained(output_dir)
model.push_to_hub(model_type[5])

05/31/2021 19:41:01 - INFO - huggingface_hub.repository -   git version 2.17.1
Sorry, no usage text found for "git-lfs"
05/31/2021 19:42:34 - INFO - huggingface_hub.repository -   
Git LFS: (0 of 1 files) 0 B / 974.07 MB                                        
Git LFS: (0 of 1 files) 0 B / 974.07 MB                                        
Git LFS: (0 of 1 files) 0 B / 974.07 MB                                        
Git LFS: (0 of 1 files) 43.45 KB / 974.07 MB                                   
Git LFS: (0 of 1 files) 103.45 KB / 974.07 MB                                  
Git LFS: (0 of 1 files) 151.45 KB / 974.07 MB                                  
Git LFS: (0 of 1 files) 267.45 KB / 974.07 MB                                  
Git LFS: (0 of 1 files) 483.45 KB / 974.07 MB                                  
Git LFS: (0 of 1 files) 1.01 MB / 974.07 MB                                    
Git LFS: (0 of 1 files) 4.94 MB / 974.07 MB                                    
Git LFS: (0 of 1 fi